# 4.7.Transfer_Learning_MobileNet2_SGD_ULTIMO_500_EPOCHS

In [ ]:
#!pip install sklearn
#!pip install imutils

In [1]:
import tensorflow as tf

from tensorflow.keras.applications import MobileNetV2

from tensorflow_examples.lite.model_maker.core.data_util.image_dataloader import ImageClassifierDataLoader

import matplotlib.pyplot as plt
import numpy as np

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Empezamos el entrenamiento

In [3]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from pyimagesearch.preprocessing import ImageToArrayPreprocessor
from pyimagesearch.preprocessing import AspectAwarePreprocessor
from pyimagesearch.datasets import SimpleDatasetLoader

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

from imutils import paths
import numpy as np
import os
import cv2

# Leemos la data

In [4]:
img_size = 224
path_train = 'data_' + str(img_size) + '/train'
path_validation = 'data_' + str(img_size) + '/validation'
path_checkpoints = 'checkpoint_MobileNet'

In [5]:
imagePaths_train = list(paths.list_images(path_train))
imagePaths_validation = list(paths.list_images(path_validation))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths_train]
classNames = [str(x) for x in np.unique(classNames)]

print("etiquetas del entrenamiento")
print(classNames)

#Códificando las etiquetas en númerops
print("codificando etiquetas")
le = LabelEncoder()
classNames_ids = le.fit_transform(classNames)
print(classNames_ids)

etiquetas del entrenamiento
['10', '100', '20', '200', '50']
codificando etiquetas
[0 1 2 3 4]


In [ ]:
iap = ImageToArrayPreprocessor()
sdl = SimpleDatasetLoader(preprocessors=[iap])

print('Cargando data para el entrenamiento')
data_train, labels_train = sdl.load(imagePaths_train, verbose=5000)

print('Cargando data para la validación')
data_validation, labels_validation = sdl.load(imagePaths_validation, verbose=500)

print('Binarizando los labels')
labels_train = LabelBinarizer().fit_transform(labels_train)
labels_validation = LabelBinarizer().fit_transform(labels_validation)

print('Carga terminada')


# Creando el modelo base

In [28]:
base_model = MobileNetV2(
    include_top=False,
    #weights=None,
    weights='imagenet',
    input_shape=(img_size,img_size,3)
)

# Congelamos las capas para el entrenamiento

In [29]:
for layer in base_model.layers:
    layer.trainable = False

# Creamos la nueva cabecera

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization

# Creamos varios modelos para luego realizar un ensamblaje

In [47]:
path_ensembles = 'ensembles_MobileNetV2'
batch_size = 32

# Entrenamiento

In [48]:
aug_train = ImageDataGenerator(rescale=1. / 255,
                               featurewise_std_normalization=True,
                               samplewise_std_normalization=True,
                               samplewise_center=True)
#aug_train.fit(data_train)
train_generator = aug_train.flow_from_directory(path_train,
                                                target_size=(img_size, img_size),
                                                batch_size=batch_size)

Found 2240 images belonging to 5 classes.


In [49]:
aug_validation = ImageDataGenerator(rescale=1. / 255)
#aug_validation.fit(data_validation)
validation_generator = aug_validation.flow_from_directory(path_validation,
                                                          target_size=(img_size, img_size),
                                                          batch_size=batch_size)

Found 2671 images belonging to 5 classes.


In [67]:
model = tf.keras.Sequential([
                              base_model,
                              #GlobalAveragePooling2D(),
                              Flatten(),
                              BatchNormalization(),
                              Dropout(0.25),
                              Dense(1024, activation='relu'),
                              BatchNormalization(),
                              Dense(1024, activation='relu'),
                              BatchNormalization(),
                              Dense(1024, activation='relu'),
                              BatchNormalization(),
                              #Dense(1024, activation='relu'),
                              Dropout(0.25),
                              #Dense(1024, activation='relu'),
                              Dense(512, activation='relu'),
                              BatchNormalization(),
                              Dense(512, activation='relu'),
                              BatchNormalization(),
                              Dense(512, activation='relu'),
                              BatchNormalization(),
                              Dropout(0.25),
                              #Dense(512, activation='relu'),
                              #Dropout(0.25),
                              Dense(128, activation='relu'),
                              BatchNormalization(),
                              Dense(128, activation='relu'),
                              BatchNormalization(),
                              Dense(128, activation='relu'),
                              BatchNormalization(),
                              Dropout(0.25),
                              #Dense(128, activation='relu'),
                              #Dropout(0.2),
                              Dense(64, activation='relu'),
                              BatchNormalization(),
                              Dense(64, activation='relu'),
                              BatchNormalization(),
                              Dense(64, activation='relu'),
                              BatchNormalization(),
                              Dropout(0.25),
                              #Dense(64, activation='relu'),
                              #Dropout(0.25),
                              #Dense(32, activation='relu'),
                              Dense(len(classNames_ids), activation='softmax')
                            ])

In [68]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
flatten_6 (Flatten)          (None, 62720)             0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 62720)             250880    
_________________________________________________________________
dropout_16 (Dropout)         (None, 62720)             0         
_________________________________________________________________
dense_97 (Dense)             (None, 1024)              64226304  
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_98 (Dense)             (None, 1024)             

In [69]:
#optimizer = Adam(learning_rate=0.0001)
optimizer = SGD(learning_rate=0.0001)

loss = 'categorical_crossentropy'
metrics=["accuracy"]

model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)


# Configurando checkpoint

In [70]:
fname = os.path.sep.join([path_checkpoints, 
                          'epoch-{epoch:03d}-val_loss:{val_loss:.4f}-val_accuracy:{val_accuracy:.4f}.hdf5'])
checkpoint = ModelCheckpoint(fname,
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min',
                             verbose=1)
#callbacks = [checkpoint]
callbacks = []

In [71]:
epochs = 3000

history = model.fit_generator(train_generator,
                              steps_per_epoch=len(train_generator),
                              validation_data=validation_generator,
                              epochs = epochs,
                              callbacks=callbacks,
                              verbose = 1)

Epoch 1/3000
70/70 [==============================] - 8s 121ms/step - loss: 2.3315 - accuracy: 0.2098 - val_loss: 1.6096 - val_accuracy: 0.2175
Epoch 2/3000
70/70 [==============================] - 8s 112ms/step - loss: 2.3256 - accuracy: 0.2058 - val_loss: 1.6145 - val_accuracy: 0.2325
Epoch 3/3000
70/70 [==============================] - 8s 113ms/step - loss: 2.2635 - accuracy: 0.2201 - val_loss: 1.6377 - val_accuracy: 0.2437
Epoch 4/3000
70/70 [==============================] - 8s 114ms/step - loss: 2.3084 - accuracy: 0.2174 - val_loss: 1.6838 - val_accuracy: 0.2422
Epoch 5/3000
70/70 [==============================] - 8s 113ms/step - loss: 2.2513 - accuracy: 0.2196 - val_loss: 1.7278 - val_accuracy: 0.2355
Epoch 6/3000
70/70 [==============================] - 8s 113ms/step - loss: 2.2689 - accuracy: 0.2246 - val_loss: 1.7676 - val_accuracy: 0.2250
Epoch 7/3000
70/70 [==============================] - 8s 113ms/step - loss: 2.1883 - accuracy: 0.2330 - val_loss: 1.7806 - val_accuracy:

70/70 [==============================] - 8s 114ms/step - loss: 1.6249 - accuracy: 0.3643 - val_loss: 1.6848 - val_accuracy: 0.2748
Epoch 58/3000
70/70 [==============================] - 8s 115ms/step - loss: 1.6111 - accuracy: 0.3683 - val_loss: 1.6871 - val_accuracy: 0.2673
Epoch 59/3000
70/70 [==============================] - 8s 114ms/step - loss: 1.6370 - accuracy: 0.3549 - val_loss: 1.6974 - val_accuracy: 0.2718
Epoch 60/3000
70/70 [==============================] - 8s 114ms/step - loss: 1.6362 - accuracy: 0.3576 - val_loss: 1.6962 - val_accuracy: 0.2726
Epoch 61/3000
70/70 [==============================] - 8s 115ms/step - loss: 1.5861 - accuracy: 0.3804 - val_loss: 1.6956 - val_accuracy: 0.2681
Epoch 62/3000
70/70 [==============================] - 8s 115ms/step - loss: 1.6023 - accuracy: 0.3585 - val_loss: 1.7016 - val_accuracy: 0.2651
Epoch 63/3000
70/70 [==============================] - 8s 115ms/step - loss: 1.5839 - accuracy: 0.3701 - val_loss: 1.6965 - val_accuracy: 0.2669

70/70 [==============================] - 8s 115ms/step - loss: 1.0712 - accuracy: 0.5830 - val_loss: 1.6721 - val_accuracy: 0.3059
Epoch 114/3000
70/70 [==============================] - 8s 115ms/step - loss: 1.0771 - accuracy: 0.5661 - val_loss: 1.6730 - val_accuracy: 0.3021
Epoch 115/3000
70/70 [==============================] - 8s 114ms/step - loss: 1.0494 - accuracy: 0.5875 - val_loss: 1.6708 - val_accuracy: 0.3029
Epoch 116/3000
70/70 [==============================] - 8s 114ms/step - loss: 1.0440 - accuracy: 0.5813 - val_loss: 1.6705 - val_accuracy: 0.3033
Epoch 117/3000
70/70 [==============================] - 8s 115ms/step - loss: 1.0453 - accuracy: 0.5821 - val_loss: 1.6669 - val_accuracy: 0.3081
Epoch 118/3000
70/70 [==============================] - 8s 116ms/step - loss: 1.0094 - accuracy: 0.6201 - val_loss: 1.6656 - val_accuracy: 0.3126
Epoch 119/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.9963 - accuracy: 0.6174 - val_loss: 1.6656 - val_accuracy: 

70/70 [==============================] - 8s 114ms/step - loss: 0.6125 - accuracy: 0.7821 - val_loss: 1.7083 - val_accuracy: 0.3201
Epoch 170/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.6248 - accuracy: 0.7732 - val_loss: 1.7085 - val_accuracy: 0.3209
Epoch 171/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.6271 - accuracy: 0.7759 - val_loss: 1.7127 - val_accuracy: 0.3212
Epoch 172/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.6157 - accuracy: 0.7768 - val_loss: 1.7150 - val_accuracy: 0.3205
Epoch 173/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.6218 - accuracy: 0.7728 - val_loss: 1.7212 - val_accuracy: 0.3179
Epoch 174/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.6134 - accuracy: 0.7737 - val_loss: 1.7264 - val_accuracy: 0.3220
Epoch 175/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.6166 - accuracy: 0.7786 - val_loss: 1.7248 - val_accuracy: 

70/70 [==============================] - 8s 114ms/step - loss: 0.4141 - accuracy: 0.8603 - val_loss: 1.8148 - val_accuracy: 0.3231
Epoch 226/3000
70/70 [==============================] - 8s 113ms/step - loss: 0.3884 - accuracy: 0.8853 - val_loss: 1.8149 - val_accuracy: 0.3261
Epoch 227/3000
70/70 [==============================] - 8s 113ms/step - loss: 0.3879 - accuracy: 0.8839 - val_loss: 1.8262 - val_accuracy: 0.3227
Epoch 228/3000
70/70 [==============================] - 8s 113ms/step - loss: 0.3937 - accuracy: 0.8763 - val_loss: 1.8237 - val_accuracy: 0.3220
Epoch 229/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.3689 - accuracy: 0.8924 - val_loss: 1.8245 - val_accuracy: 0.3205
Epoch 230/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.3828 - accuracy: 0.8804 - val_loss: 1.8260 - val_accuracy: 0.3194
Epoch 231/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.3846 - accuracy: 0.8804 - val_loss: 1.8239 - val_accuracy: 

...

Epoch 2689/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.0268 - accuracy: 0.9924 - val_loss: 2.7241 - val_accuracy: 0.4212
Epoch 2690/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0233 - accuracy: 0.9937 - val_loss: 2.7313 - val_accuracy: 0.4174
Epoch 2691/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0171 - accuracy: 0.9964 - val_loss: 2.7000 - val_accuracy: 0.4216
Epoch 2692/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0223 - accuracy: 0.9955 - val_loss: 2.6853 - val_accuracy: 0.4216
Epoch 2693/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0163 - accuracy: 0.9951 - val_loss: 2.7414 - val_accuracy: 0.4171
Epoch 2694/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0279 - accuracy: 0.9924 - val_loss: 2.7462 - val_accuracy: 0.4167
Epoch 2695/3000
70/70 [==============================] - 8s 113ms/step - loss: 0.0162 - accuracy: 0.9951 - val_loss: 2

Epoch 2745/3000
70/70 [==============================] - 8s 113ms/step - loss: 0.0240 - accuracy: 0.9951 - val_loss: 2.7156 - val_accuracy: 0.4219
Epoch 2746/3000
70/70 [==============================] - 8s 113ms/step - loss: 0.0182 - accuracy: 0.9955 - val_loss: 2.7513 - val_accuracy: 0.4171
Epoch 2747/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.0117 - accuracy: 0.9969 - val_loss: 2.7430 - val_accuracy: 0.4178
Epoch 2748/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0152 - accuracy: 0.9955 - val_loss: 2.7118 - val_accuracy: 0.4208
Epoch 2749/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0106 - accuracy: 0.9987 - val_loss: 2.7196 - val_accuracy: 0.4189
Epoch 2750/3000
70/70 [==============================] - 8s 112ms/step - loss: 0.0112 - accuracy: 0.9982 - val_loss: 2.7035 - val_accuracy: 0.4208
Epoch 2751/3000
70/70 [==============================] - 8s 113ms/step - loss: 0.0165 - accuracy: 0.9960 - val_loss: 2

Epoch 2801/3000
70/70 [==============================] - 8s 113ms/step - loss: 0.0119 - accuracy: 0.9973 - val_loss: 2.7187 - val_accuracy: 0.4189
Epoch 2802/3000
70/70 [==============================] - 8s 113ms/step - loss: 0.0191 - accuracy: 0.9964 - val_loss: 2.7201 - val_accuracy: 0.4201
Epoch 2803/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0156 - accuracy: 0.9969 - val_loss: 2.6932 - val_accuracy: 0.4227
Epoch 2804/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0169 - accuracy: 0.9978 - val_loss: 2.6494 - val_accuracy: 0.4264
Epoch 2805/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.0172 - accuracy: 0.9969 - val_loss: 2.7166 - val_accuracy: 0.4189
Epoch 2806/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.0163 - accuracy: 0.9969 - val_loss: 2.7014 - val_accuracy: 0.4204
Epoch 2807/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0080 - accuracy: 0.9987 - val_loss: 2

Epoch 2857/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0171 - accuracy: 0.9969 - val_loss: 2.7745 - val_accuracy: 0.4163
Epoch 2858/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0132 - accuracy: 0.9955 - val_loss: 2.7765 - val_accuracy: 0.4163
Epoch 2859/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.0160 - accuracy: 0.9951 - val_loss: 2.7778 - val_accuracy: 0.4145
Epoch 2860/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0123 - accuracy: 0.9964 - val_loss: 2.7297 - val_accuracy: 0.4186
Epoch 2861/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0126 - accuracy: 0.9978 - val_loss: 2.7330 - val_accuracy: 0.4186
Epoch 2862/3000
70/70 [==============================] - 8s 115ms/step - loss: 0.0122 - accuracy: 0.9969 - val_loss: 2.7578 - val_accuracy: 0.4171
Epoch 2863/3000
70/70 [==============================] - 8s 114ms/step - loss: 0.0109 - accuracy: 0.9987 - val_loss: 2

KeyboardInterrupt: 

In [ ]:
#guardando el modelo
p = [path_ensembles, "model_500_SGD.model"]
model.save(os.path.sep.join(p))

In [ ]:
#evaluando el modelo
predictions = model.predict(data_validation, batch_size=batch_size)
print(classification_report(labels_validation.argmax(axis=1),
                            predictions.argmax(axis=1),
                            target_names=classNames))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
#plt.ylim([min(plt.ylim()),1])
plt.ylim([0,1.1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
#plt.ylim([0,1.0])
plt.ylim([0,20])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# Tomando 25 imágenes de validación y observando el resultado

In [ ]:
test_data = ImageClassifierDataLoader.from_folder(path_validation)

def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

plt.figure(figsize=(20, 20))

for i, (image, label) in enumerate(test_data.dataset.take(100)):
    ax = plt.subplot(10, 10, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image.numpy(), cmap=plt.cm.gray)
    
    # predict
    swapped = np.moveaxis(image, 0, 1)
    arr4d = np.expand_dims(swapped, 0)
    id_predict = np.argmax(model.predict(arr4d))
    predict_label = classNames[id_predict]
    
    color = get_label_color(predict_label, test_data.index_to_label[label.numpy()])
    #color = 'black'
    ax.xaxis.label.set_color(color)
    plt.xlabel('Predicted: %s' % predict_label)
plt.show()